In [13]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time
import pandas as pd
import numpy as np
from datetime import date

executable_path = {"executable_path":"chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=True)

# URL of page to be scraped
url = 'http://www.licigob.com.mx/cgi-bin/cupisa_optodhue93hefg8yu4iewfrgh.asp'

# 

time.sleep(5)
response = browser.html
soup = bs(response.text, 'html.parser')

results = soup.find_all('table')
allrows = results[3].find_all("tr")
test_rows = allrows[1:]
licitaciones = []
row_counter = 0
concurso_dict = {}
for rows in test_rows:
    data = rows.find_all("td")
    firsttext = data[0].text
    if firsttext == "Dependencia":
        pass
    elif firsttext == "Bases":
        link = rows.find("a")["href"]
        concurso_dict["link"] = link
    elif len(data) > 3:
        concurso_dict["interno"] = "CUPISA"
        concurso_dict["Dependencia"] = data[0].text
        concurso_dict["Lugar"] = data[1].text
        if data[2].text.strip() == "":
            fechaactual = date.today().strftime("%d/%m/%Y")
            concurso_dict["fecha_de_publicacion"] = fechaactual
        else:
            concurso_dict["fecha_de_publicacion"] = data[2].text.strip()
        concurso_dict["fecha_de_presentacion"] = data[3].text
        concurso_dict["Modalidad"] = data[4].text
        concurso_dict["Numero_de_procedimiento"] = data[5].text
    elif len(data) == 1:
        concurso_dict["Descripcion"] = data[0].text
        licitaciones.append(concurso_dict)
        concurso_dict = {}


url = 'http://www.licigob.com.mx/cgi-bin/infineo_todfh298309qewtd672fgy354wte.asp'

response = requests.get(url, timeout=(6, 6))
time.sleep(5)
response = browser.html
soup = bs(response.text, 'html.parser')

results = soup.find_all('table')
allrows = results[3].find_all("tr")
test_rows = allrows[1:]
row_counter = 0
concurso_dict = {}
for rows in test_rows:
    data = rows.find_all("td")
    firsttext = data[0].text
    if firsttext == "Dependencia":
        pass
    elif firsttext == "Bases":
        link = rows.find("a")["href"]
        concurso_dict["link"] = link
    elif len(data) > 3:
        concurso_dict["interno"] = "URALES"
        concurso_dict["Dependencia"] = data[0].text
        concurso_dict["Lugar"] = data[1].text
        if data[2].text.strip() == "":
            fechaactual = date.today().strftime("%d/%m/%Y")
            concurso_dict["fecha_de_publicacion"] = fechaactual
        else:
            concurso_dict["fecha_de_publicacion"] = data[2].text.strip()
        concurso_dict["fecha_de_presentacion"] = data[3].text
        concurso_dict["Modalidad"] = data[4].text
        concurso_dict["Numero_de_procedimiento"] = data[5].text
    elif len(data) == 1:
        concurso_dict["Descripcion"] = data[0].text
        licitaciones.append(concurso_dict)
        concurso_dict = {}





In [14]:
df = pd.DataFrame(licitaciones)
df["unique"] = df["Numero_de_procedimiento"].duplicated()
df.loc[df["unique"] == True, "interno"] = "COMBINADO"
df.drop("unique", axis=1, inplace=True)
df_nodup=df.drop_duplicates("Numero_de_procedimiento", keep="last")
df_nodup["fecha_de_publicacion"] = pd.to_datetime(df_nodup["fecha_de_publicacion"], dayfirst=True,  format="%d/%m/%Y")
df_nodup["fecha_de_presentacion"] = pd.to_datetime(df_nodup["fecha_de_presentacion"], dayfirst=True,  format="%d/%m/%Y")
df_nodup.sort_values(by=["fecha_de_publicacion"], ascending=False ,inplace=True)

In [15]:
df_nodup["Nuevo"] = "No"

In [16]:
from datetime import datetime, timedelta
df_nodup.loc[df_nodup["fecha_de_publicacion"] >= pd.to_datetime(date.today() - timedelta(days=2)), "Nuevo"] = "Si"

In [17]:
df_nodup

,interno,Dependencia,Lugar,fecha_de_publicacion,fecha_de_presentacion,Modalidad,Numero_de_procedimiento,link,Descripcion,Nuevo
0,CUPISA,Gobierno del Estado de Jalisco,Jalisco,2020-06-16,2020-06-23,Licitación Pública,SIOP-E-SMA-OB-CSS-301-2020,/cgi-bin/pdf/nodisponible.html,"Construcción del Centro de Salud Tala, CLUES J...",Si
1,URALES,Gobierno del Estado de Jalisco,Jalisco,2020-06-16,2020-06-23,Licitación Pública,SIOP-E-SMA-OB-CSS-249-2020,/cgi-bin/pdf/nodisponible.html,"Rehabilitación del Hospital Regional de Ameca,...",Si


In [18]:
antier= date.today() - timedelta(days=2)
pd.to_datetime(antier)

Timestamp('2020-06-14 00:00:00')